![Cabec%CC%A7alho_notebook.png](cabecalho_notebook.png)

# PCA - Tarefa 01: *HAR* com PCA

Vamos trabalhar com a base da demonstração feita em aula, mas vamos explorar um pouco melhor como é o desempenho da árvore variando o número de componentes principais.

**Fonte:** https://archive.ics.uci.edu/dataset/240/human+activity+recognition+using+smartphones

In [33]:
import pandas as pd

from sklearn.tree import DecisionTreeClassifier

from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

In [34]:
filename_features = "Dados/UCI HAR Dataset/features.txt"
filename_labels = "Dados/UCI HAR Dataset/activity_labels.txt"

filename_subtrain = "Dados/UCI HAR Dataset/train/subject_train.txt"
filename_xtrain = "Dados/UCI HAR Dataset/train/X_train.txt"
filename_ytrain = "Dados/UCI HAR Dataset/train/y_train.txt"

filename_subtest = "Dados/UCI HAR Dataset/test/subject_test.txt"
ffilename_xtest = "Dados/UCI HAR Dataset/test/X_test.txt"
filename_ytest = "Dados/UCI HAR Dataset/test/y_test.txt"

features = pd.read_csv(filename_features, header=None, names=['nome_var'], sep="#").squeeze()
labels = pd.read_csv(filename_labels, delim_whitespace=True, header=None, names=['cod_label', 'label'])

subject_train = pd.read_csv(filename_subtrain, header=None, names=['subject_id']).squeeze()
X_train = pd.read_csv(filename_xtrain, delim_whitespace=True, header=None, names=features.tolist())
y_train = pd.read_csv(filename_ytrain, header=None, names=['cod_label'])

subject_test = pd.read_csv(filename_subtest, header=None, names=['subject_id']).squeeze()
X_test = pd.read_csv(ffilename_xtest, delim_whitespace=True, header=None, names=features.tolist())
y_test = pd.read_csv(filename_ytest, header=None, names=['cod_label'])

## Árvore de decisão

Rode uma árvore de decisão com todas as variáveis, utilizando o ```ccp_alpha=0.001```. Avalie a acurácia nas bases de treinamento e teste. Avalie o tempo de processamento.

In [35]:
%%time

ccp_alpha = 0.001
clf = DecisionTreeClassifier(random_state = 2360873, ccp_alpha = ccp_alpha, min_samples_leaf = 20).fit(X_train, y_train)

train_accuracy = accuracy_score(y_train, clf.predict(X_train))
test_accuracy = accuracy_score(y_test, clf.predict(X_test))

print(f'Acurácia no treinamento: {train_accuracy:.2f}')
print(f'Acurácia no teste: {test_accuracy:.2f}')
print('\nTempo de Processamento:')

Acurácia no treinamento: 0.96
Acurácia no teste: 0.87

Tempo de Processamento:
CPU times: total: 4.97 s
Wall time: 4.97 s


## Árvore com PCA

Faça uma análise de componemtes principais das variáveis originais. Utilize apenas uma componente. Faça uma árvore de decisão com esta componente como variável explicativa.

- Avalie a acurácia nas bases de treinamento e teste
- Avalie o tempo de processamento

In [36]:
%%time

prcomp = PCA(n_components=1).fit(X_train)

pc_treino = prcomp.transform(X_train)
pc_valida = prcomp.transform(X_valid)
pc_teste  = prcomp.transform(X_test)

clf = DecisionTreeClassifier(random_state = 2360873, min_samples_leaf=20).fit(pc_treino, y_train)

train_accuracy = accuracy_score(y_train, clf.predict(pc_treino))
valid_accuracy = accuracy_score(y_valid, clf.predict(pc_valida))
test_accuracy = accuracy_score(y_test, clf.predict(pc_teste))

print(f'Acurácia no treinamento: {train_accuracy:.2f}')
print(f'Acurácia na validação: {valid_accuracy:.2f}')
print(f'Acurácia no teste: {test_accuracy:.2f}')
print('\nTempo de Processamento:')

Acurácia no treinamento: 0.56
Acurácia na validação: 0.56
Acurácia no teste: 0.44

Tempo de Processamento:
CPU times: total: 781 ms
Wall time: 342 ms


## Testando o número de componentes

Com base no código acima, teste a árvore de classificação com pelo menos as seguintes possibilidades de quantidades de componentes: ```[1, 2, 5, 10, 50]```. Avalie para cada uma delas:

- Acurácia nas bases de treino e teste
- Tempo de processamento


In [39]:
%%time

n_components_list = [1, 2, 5, 10, 50]

resultados = {}
for n_components in n_components_list:

    pca = PCA(n_components = n_components).fit(X_train)
    
    pc_treino = pca.transform(X_train)
    pc_valida = prcomp.transform(X_valid)
    pc_teste  = pca.transform(X_test)
    
    clf = DecisionTreeClassifier(random_state=2360873, min_samples_leaf=20).fit(pc_treino, y_train)

    train_accuracy = accuracy_score(y_train, clf.predict(pc_treino))
    test_accuracy = accuracy_score(y_test, clf.predict(pc_teste))
    
    resultados[n_components] = {
        'train_accuracy': train_accuracy,
        'test_accuracy': test_accuracy}

for n_components, nome in resultados.items():
    print(f'Nº de Componentes: {n_components}')
    print(f'Acurácia no treinamento: {nome["train_accuracy"]:.2f}')
    print(f'Acurácia no teste: {nome["test_accuracy"]:.2f}')
    print('-'*20)

print('\nTempo de Processamento:')

Nº de Componentes: 1
Acurácia no treinamento: 0.56
Acurácia no teste: 0.44
--------------------
Nº de Componentes: 2
Acurácia no treinamento: 0.67
Acurácia no teste: 0.56
--------------------
Nº de Componentes: 5
Acurácia no treinamento: 0.87
Acurácia no teste: 0.78
--------------------
Nº de Componentes: 10
Acurácia no treinamento: 0.90
Acurácia no teste: 0.82
--------------------
Nº de Componentes: 50
Acurácia no treinamento: 0.91
Acurácia no teste: 0.81
--------------------

Tempo de Processamento:
CPU times: total: 6.16 s
Wall time: 2.63 s


## Conclua

- O que aconteceu com a acurácia?
- O que aconteceu com o tempo de processamento?

**Resposta**

- No último exercício, quanto maior é o número de **Componentes**, mais elevada é a acurácia.
- Os três códigos teve tempo de processamento diferentes, o 3 foi o com maior tempo.